In [16]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [17]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-28 04:05:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.19MB/s    in 0.2s    

2022-01-28 04:05:24 (5.19 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [18]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LawnGarden").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [19]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [5]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [23]:
df = df.dropna()
df = df.dropDuplicates()

In [22]:
from pyspark.sql.functions import to_date
df = df.withColumn("customer_id",df.customer_id.cast('int'))
df = df.withColumn("product_parent",df.product_parent.cast('int'))
df = df.withColumn("star_rating",df.star_rating.cast('int'))
df = df.withColumn("helpful_votes",df.helpful_votes.cast('int'))
df = df.withColumn("total_votes",df.total_votes.cast('int'))
df = df.withColumn('review_date',to_date(df.review_date, 'yyyy-MM-dd'))


In [21]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [20]:
review_id_table_df = df.select(["review_id","customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df = review_id_table_df.dropDuplicates(['review_id'])


In [11]:
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates(['product_id'])

In [12]:
customers_df = df.groupBy("customer_id").count()
customers_df = customers_df.withColumnRenamed('count', 'customer_count')

In [13]:
vine_table_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table_df = vine_table_df.dropDuplicates(['review_id'])

In [14]:
 # Configuration for RDS instance
mode = "append"
jdbc_url="jdbc:postgresql://datavis-games-db.ci2abqkevhbv.us-west-1.rds.amazonaws.com:5432/games_db"
config = {"user":"<user_name>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [17]:
# Write DataFrame to table
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_lawn', mode=mode, properties=config) 


In [18]:
products_df.write.jdbc(url=jdbc_url, table='products_lawn', mode=mode, properties=config) 


In [19]:
customers_df.write.jdbc(url=jdbc_url, table='customers_lawn', mode=mode, properties=config) 


In [15]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_lawn', mode=mode, properties=config) 